In [2]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [3]:
red_apples = []
green_apples = []

for directory in os.listdir(r"C:\Python_Projects\esaditional_GAN\conditional-GAN\datasett"):
    path = os.path.join(r"C:\Python_Projects\esaditional_GAN\conditional-GAN\datasett", directory)
    for image_name in os.listdir(path):
        image_path = os.path.join(path, image_name)
        image = Image.open(image_path)
        image = image.convert("RGB")
        if (directory == "red_apples"):
            img = np.asarray(image)
            red_apples.append(img)
        elif (directory == "green_apples"):
            img = np.asarray(image)
            green_apples.append(img)

In [4]:
a = red_apples[1]

a.shape

(300, 300, 3)

### Generator and Discriminator blocks

In [ ]:
class Generator(nn.Module):
    def __init__(self, 
                input_channels,
                encode_channel_sizes,
                encode_kernel_sizes, 

                decode_channel_sizes, 
                decode_kernel_sizes,
                 
                dropout, 

                noise_size,                  
                sat_im_size) -> None:
        
        self.input_channels: int = input_channels
        self.encode_channel_sizes: list = encode_channel_sizes
        self.encode_kernel_sizes: list = encode_kernel_sizes

        self.decode_channel_sizes: list = decode_channel_sizes
        self.decode_kernel_sizes: list = decode_kernel_sizes

        self.dropout: float = dropout

        super().__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(self.input_channels, self.encode_channel_sizes[0], self.encode_kernel_sizes[0]),
            nn.Conv2d(self.encode_channel_sizes[0], self.encode_channel_sizes[0], self.encode_channel_sizes[0]),
            nn.MaxPool2d(kernel_size= 2, stride= 2),
            nn.Dropout(self.dropout),

            nn.Conv2d(self.encode_channel_sizes[0], self.encode_channel_sizes[1], self.encode_kernel_sizes[1]),
            nn.Conv2d(self.encode_channel_sizes[1], self.encode_channel_sizes[1], self.encode_kernel_sizes[1]),
            nn.MaxPool2d(kernel_size= 2, stride= 2),
            nn.Dropout(self.dropout),    

            nn.Conv2d(self.encode_channel_sizes[1], self.encode_channel_sizes[2], self.encode_kernel_sizes[2]),
            nn.Conv2d(self.encode_channel_sizes[2], self.encode_channel_sizes[2], self.encode_kernel_sizes[2]),
            nn.MaxPool2d(kernel_size= 2, stride= 2),
            nn.Dropout(self.dropout),

            #here may come some ANN layers 
            #idk why :D
            
            nn.ConvTranspose2d(self.encode_channel_sizes[2], self.decode_channel_sizes[0], self.decode_kernel_sizes[0]),
            nn.Conv2d(self.decode_channel_sizes[0], self.decode_channel_sizes[0], self.decode_kernel_sizes[0]),
            nn.Conv2d(self.decode_channel_sizes[0], self.decode_channel_sizes[0], self.decode_kernel_sizes[0]),
            nn.Dropout(self.dropout),

            nn.ConvTranspose2d(self.decode_channel_sizes[0], self.decode_channel_sizes[1], self.decode_kernel_sizes[1]),
            nn.Conv2d(self.decode_channel_sizes[1], self.decode_channel_sizes[1], self.decode_kernel_sizes[1]),
            nn.Conv2d(self.decode_channel_sizes[1], self.decode_channel_sizes[1], self.decode_kernel_sizes[1]),
            nn.Dropout(self.dropout),

            nn.ConvTranspose2d(self.decode_channel_sizes[1], self.decode_channel_sizes[2], self.decode_kernel_sizes[2]),
            nn.Conv2d(self.decode_channel_sizes[2], self.decode_channel_sizes[2], self.decode_kernel_sizes[2]),
            nn.Conv2d(self.decode_channel_sizes[2], self.decode_channel_sizes[2], self.decode_kernel_sizes[2]),
            nn.Dropout(self.dropout)
            )
        
    def forward(self, noise: torch.Tensor, lable_image: torch.Tensor):
        
        x = torch.cat([noise, lable_image], 2)

        out = self.model(x)

        return out

In [1]:
class Discriminator(nn.Module):
    def __init__(self,
                 input_chanel,
                 channel_sizes,
                 kernel_sizes,
                 
                 ann_layer_sizes,
                 ) -> None:
        
        self.input_channel: int = input_chanel
        self.channel_sizes: list = channel_sizes
        self.kernel_sizes: list  = kernel_sizes

        self.ann_layer_sizes: list = ann_layer_sizes

        super().__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(self.input_channel, self.channel_sizes[0], self.kernel_sizes[0]),
            nn.Conv2d(self.channel_sizes[0], self.channel_sizes[0], self.kernel_sizes[0]),
            nn.MaxPool2d(kernel_size= 2, stride= 2),    

            nn.Conv2d(self.channel_sizes[1], self.channel_sizes[1], self.kernel_sizes[1]),
            nn.Conv2d(self.channel_sizes[1], self.channel_sizes[1], self.kernel_sizes[1]),
            nn.MaxPool2d(kernel_size= 2, stride= 2),   

            nn.Conv2d(self.channel_sizes[2], self.channel_sizes[2], self.kernel_sizes[2]),
            nn.Conv2d(self.channel_sizes[2], self.channel_sizes[2], self.kernel_sizes[2]),
            nn.MaxPool2d(kernel_size= 2, stride= 2),
            
            nn.Flatten(),

            nn.Linear(self.ann_layer_sizes[0], self.ann_layer_sizes[1]),
            nn.ReLU(),
            nn.Linear(self.ann_layer_sizes[1], self.ann_layer_sizes[2]),    
            nn.ReLU(),
            nn.Linear(self.ann_layer_sizes[2], self.ann_layer_sizes[3]),
            nn.ReLU(),
            nn.Linear(self.ann_layer_sizes[2], 1),
            nn.Sigmoid()

        )
    
    def forward(self, real_image, generated_image):
        x = torch.cat([real_image, generated_image], 2)
        out = self.model(x)
        return out

    #its gonna be sequnce of conv2d layers an dense layersc  
    #conv2d will demolishing the image then dense layers with 1 output layer proces the image
    #probably :D

NameError: name 'nn' is not defined

### Train Function

In [ ]:
# Loss Function

criterion = nn.BCELoss()

In [ ]:
#Optimizer

generator_optimizer = torch.optim.Adam(Generator.parameters)
discriminator_optimizer = torch.optim.Adam(Generator.parameters)

In [ ]:
def Generator_Train():
    return None